In [7]:
import pandas as pd
import re
import os
import json
from spacy.lang.en import English # updated
import spacy
from collections import Counter
from tqdm.notebook import tqdm

In [12]:
df = pd.read_csv('/shared/0/projects/news-quotes/reg_data_plot.csv')

In [14]:
df.columns

Index(['url', 'doi', 'author_id', 'affiliation_ids', 'author_seq_num',
       'author_name', 'author_pos_cate', 'author_last_name',
       'last_name_length', 'last_name_prob', 'author_eth_ethnea',
       'author_gender_ethnea', 'author_eth_wiki', 'author_eth_census',
       'reporter_name', 'reporter_eth_ethnea', 'reporter_gender_ethnea',
       'reporter_eth_wiki', 'reporter_eth_census', 'author_eth_ethnea_broad',
       'reporter_eth_ethnea_broad', 'author_eth_wiki_broad',
       'reporter_eth_wiki_broad', 'is_top_author', 'doi_men_cn', 'author_rank',
       'affiliation_cate', 'affiliation_rank', 'journal_title',
       'journal_impact', 'top_journal', 'num_authors', 'outlet', 'category',
       'num_words', 'num_mentioned_papers', 'mention_year',
       'mention_year_center', 'gap_in_years', 'is_author_mentioned',
       'FleschReadingEase', 'sentences_per_paragraph', 'type_token_ratio',
       'Publication Date', 'Mention Date', 'is_corresponding',
       'affiliation_name'],
   

In [5]:
urls_text = {}

with open('/shared/0/projects/news-quotes/crawl_news/url_text_clean.json', 'r') as ofile:
    for row in ofile:
        row = json.loads(row)
        url, text = row['url'], row['text']
        urls_text[url] = text
print(len(urls_text))

520061


In [8]:
nlp = spacy.load('en_core_web_sm')

In [16]:
keyword_counts = Counter()
    
def has_indirect_mention(text, univ):
    #print("LAST NAME: '%s'" % fa_last_name)
    
    
    # See if we can find a "said"-like verb to indicate they gave the quote
    has_mention = False
    has_mention_weaker = False
    
    univ_keywords = re.split(r"\W+", univ)
    blacklist = set(['University', 'the', 'Technology', 'of', 'Research', 'for', 
                     'Institute', 'Medical', 's', 'and', 'National', 'Center',
                    'College', 'State', 'at'])
    univ_keywords = [x for x in univ_keywords if x not in blacklist] 
    for kw in univ_keywords:
        keyword_counts[kw] += 1
    
    #if 'Chinese sex wizards' in text:
    #    print(text)
    
    if False:
        sentences = [i for i in nlp(text).sents]
        for sent in sentences:

            if has_mention or has_mention_weaker:
                break

            sent_text = ' '.join([t.text for t in sent])
            #print(univ, sent_text)
            if univ in sent_text:
                has_mention = True
            if first_word in sent_text:
                has_mention_weaker = True
    else:
        if univ in text:
            has_mention = True
            i = text.find(univ)
            sent_text = ' '.join(text[i-50:i+50+len(univ)].split())
        for keyword in univ_keywords:
            if keyword in text:
                has_mention_weaker = True
                i = text.find(keyword)
                sent_text = ' '.join(text[i-20:i+20+len(keyword)].split())
                break
                
    if has_mention:
        #print("Found a indirect mention of '%s':\n  %s\n" % (univ, sent_text))
        pass
    elif has_mention_weaker:
        #print("Found a POSSIBLE indirect mention of '%s' via '%s':\n  %s\n" % (univ, keyword, sent_text))
        pass            
    else:
        #print("NO QUOTE by %s:\n  %s\n" % (fa_last_name, " ".join(str(sent_text).split())))
        pass


    return has_mention # or has_mention_weaker
            

has_indirect_mention_col = []
im_count = 0

if True:
    for ri, row in df.iterrows():
        url = row['url']
        text = urls_text[url]
        univs = row['affiliation_name'].split('|')
        him = False
        # if any one of univs is mentioned, it is positive.
        for university in univs:
            if has_indirect_mention(text, university):
                him = True
                break
        has_indirect_mention_col.append(him)
        if him:
            im_count += 1
        if ri % 50000 == 0:
            print(im_count, ri, keyword_counts.most_common(10), '\n')
            #break

1 0 [('Cambridge', 1)] 

20712 50000 [('California', 3306), ('Hospital', 1776), ('Harvard', 1568), ('Health', 1490), ('London', 1242), ('San', 1117), ('Washington', 1034), ('Pennsylvania', 941), ('Johns', 845), ('Hopkins', 845)] 

41598 100000 [('California', 6382), ('Hospital', 3488), ('Harvard', 3216), ('Health', 2993), ('London', 2376), ('San', 2125), ('Washington', 1900), ('Pennsylvania', 1883), ('Science', 1754), ('Johns', 1638)] 

62622 150000 [('California', 9268), ('Hospital', 5100), ('Harvard', 4845), ('Health', 4608), ('London', 3666), ('San', 3087), ('Washington', 3045), ('Pennsylvania', 2680), ('Science', 2619), ('Texas', 2482)] 

82981 200000 [('California', 12222), ('Hospital', 6627), ('Harvard', 6437), ('Health', 6075), ('London', 4901), ('San', 4162), ('Washington', 3921), ('Pennsylvania', 3583), ('Science', 3559), ('New', 3326)] 

94488 250000 [('California', 15241), ('Harvard', 8422), ('Hospital', 8287), ('Health', 7648), ('London', 6438), ('San', 5152), ('Washington'

In [18]:
df['mentions_author_institution'] = has_indirect_mention_col

In [19]:
him_col = []
for i, row in df.iterrows():
    him = (row['is_author_mentioned'] < 1) and row['mentions_author_institution'] 
    him_col.append(him)
df['has_indirect_mention'] = him_col

In [20]:
df[['url', 'has_indirect_mention', 'mentions_author_institution']].to_csv('/shared/0/projects/news-quotes/crawl_news/has_indirect_quote.csv', index=False)
